In [1]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import load_model
from pathlib import Path

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-31 14:31:14.594536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']
NEW_DATA = ['SmallCap', 'Russell2000', 'Treasury']

labels = pd.read_csv('transfer_labels.csv')

In [13]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)

In [3]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_45131/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [4]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [5]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    2100
0    1800
Name: Label, dtype: int64


In [6]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                  Image  TimePrediction  \
0     images/OHLC/SmallCap_2019-02-01 00:00:00_14_cr...               5   
1005  images/ColoredOHLC/SmallCap_2019-02-01 00:00:0...               5   
1006  images/ColoredOHLC/SmallCap_2019-02-01 00:00:0...              30   
1063  images/ColoredOHLC/SmallCap_2019-02-01 00:00:0...               5   
1064  images/ColoredOHLC/SmallCap_2019-02-01 00:00:0...              30   

      LastPrice  FuturePrice  Label  \
0     76.620003    76.860001      1   
1005  76.620003    76.860001      1   
1006  76.620003    79.650002      1   
1063  76.620003    76.860001      1   
1064  76.620003    79.650002      1   

                                            Image_Array       Date  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
1005  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
1006  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
1063  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Evaluating model predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")

                # Filter the data
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(img_type)) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)

                # Load the model
                model_filename = f"models/{img_type}_{timeframe}_{prediction}.h5"
                #model_filename = f"models/combined_{timeframe}_{prediction}.h5"
                model = load_model(model_filename)

                X = np.array(data['Image_Array'].tolist())
                lastPrice = np.array(data['LastPrice'].tolist())
                futurePrice = np.array(data['FuturePrice'].tolist())
                y = data['Label']

                # Evaluate the model on test data
                y_pred = model.predict(X)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)
                
                accuracy = accuracy_score(y, y_pred_binary)
                precision = precision_score(y, y_pred_binary)
                recall = recall_score(y, y_pred_binary)
                f1_score = fbeta_score(y, y_pred_binary, beta=1)

                y_test_array = y.values.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                # Calculate Rate of Return RoR
                ror = np.where(y_pred_binary == 1, ((futurePrice - lastPrice)/lastPrice), 0)
                # Drop all 0s from the array
                ror = ror[ror != 0]
                # Calculate average RoR
                avg_ror = np.mean(ror)
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average RoR: {avg_ror}")
                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Average_RoR': avg_ror
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('transfer_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'tansfer_evaluation_scores.csv'.")

In [ ]:
evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Evaluating model predicting {prediction} days ahead with {timeframe} days timeframe.")

            # Filter the data
            data = labels[(labels['TimePrediction'] == prediction) &
                            (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)

            # Start at same index as test data
            split_index = int(len(data) * 0.7)
            data = data[split_index:]

            # Load the model
            model_filename = f"models/combined_{timeframe}_{prediction}.h5"
            model = load_model(model_filename)

            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = np.array(data['LastPrice'].tolist())
            futurePrice = np.array(data['FuturePrice'].tolist())
            y = data['Label']

            # Evaluate the model on test data
            y_pred = model.predict(X)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)
            
            accuracy = accuracy_score(y, y_pred_binary)
            precision = precision_score(y, y_pred_binary)
            recall = recall_score(y, y_pred_binary)
            f1_score = fbeta_score(y, y_pred_binary, beta=1)

            y_test_array = y.values.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            # Calculate Rate of Return RoR
            ror = np.where(y_pred_binary == 1, ((futurePrice - lastPrice)/lastPrice), 0)
            # Drop all 0s from the array
            ror = ror[ror != 0]
            # Calculate average RoR
            avg_ror = np.mean(ror)
            
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            print(f"Hit Rate: {hit_rate}")
            print(f"Average RoR: {avg_ror}")
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Average_RoR': avg_ror
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('transfer_combined_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'transfer_combined_evaluation_scores.csv'.")

In [7]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order

            # Load model for timeframe and prediction
            model = load_model(f"models/combined_{timeframe}_{prediction}.h5")

            # Create a new array to store the combined predictions
            y_pred_combined = []
            y_combined = []

            for date in data['Date'].unique():
                for etf in NEW_DATA:
                    new_data = data[(data['Date']==date) & (data['Image'].str.contains(f'{etf}_'))]
                    new_data = data.reset_index(drop=True)
                
                    X = np.array(new_data['Image_Array'].tolist())/255.0
                    # y equals the first label in the DataFrame
                    y = new_data['Label'].iloc[0]
                    y_combined.append(y)
                    lastPrice = new_data['LastPrice'].iloc[0]
                    futurePrice = new_data['FuturePrice'].iloc[0]
                    
                    # Evaluate the model
                    y_pred = model.predict(X)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                    # If y_pred_binary contains more 1s than 0s, then predict 1, else predict 0
                    if np.sum(y_pred_binary) > len(y_pred_binary) / 2:
                        y_pred_combined.append(1)
                    else:
                        y_pred_combined.append(0)
                
            accuracy = accuracy_score(y_combined, y_pred_combined)
            precision = precision_score(y_combined, y_pred_combined)
            recall = recall_score(y_combined, y_pred_combined)
            f1_score = fbeta_score(y_combined, y_pred_combined, beta=1)

            y_array = np.array(y_combined)
            correct_predictions = np.sum(np.array(y_pred_combined) == y_array)
            hit_rate = correct_predictions / len(y_array)

            # Calculate Rate of Return RoR
            ror = np.where(y_pred_combined == 1, ((futurePrice - lastPrice)/lastPrice), 0)
            # Drop all 0s from the array
            ror = ror[ror != 0]
            # Calculate average RoR
            avg_ror = np.mean(ror)
        
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Average_RoR': avg_ror
            }, ignore_index=True)
        
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_prediction_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_prediction_evaluation_scores.csv'.")

Predicting 5 days ahead with 14 days timeframe.
12/12 [==============================] - 2s 129ms/step
Evaluation Metrics:
Accuracy: 0.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
--------------------------------------------------
Predicting 5 days ahead with 30 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


11/11 [==============================] - 4s 392ms/step
Evaluation Metrics:
Accuracy: 0.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
--------------------------------------------------
Predicting 5 days ahead with 90 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


 1/11 [=>............................] - ETA: 8s

KeyboardInterrupt: 